# Talking Head(?) Anime from a Single Image 3: Now the Body Too (Manual Poser Tool)

**Instruction**

1. Run the four cells below, one by one, in order by clicking the "Play" button to the left of it. Wait for each cell to finish before going to the next one.
2. Scroll down to the end of the last cell, and play with the GUI.

**Links**

* Github repository: http://github.com/pkhungurn/talking-head-anime-3-demo
* Project writeup: http://pkhungurn.github.io/talking-head-anime-3/

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Clone the repository
%cd /content/drive/MyDrive/diffusion
# !git clone https://github.com/pkhungurn/talking-head-anime-3-demo.git

/content/drive/MyDrive/diffusion


In [3]:
# CD into the repository directory.
%cd /content/drive/MyDrive/diffusion/talking-head-anime-3-demo

/content/drive/MyDrive/diffusion/talking-head-anime-3-demo


In [4]:
# # Download model files
# !mkdir -p data/models/standard_float
# !wget -O data/models/standard_float/editor.pt https://www.dropbox.com/s/zp3e5ox57sdws3y/editor.pt?dl=0
# !wget -O data/models/standard_float/eyebrow_decomposer.pt https://www.dropbox.com/s/bcp42knbrk7egk8/eyebrow_decomposer.pt?dl=0
# !wget -O data/models/standard_float/eyebrow_morphing_combiner.pt https://www.dropbox.com/s/oywaiio2s53lc57/eyebrow_morphing_combiner.pt?dl=0
# !wget -O data/models/standard_float/face_morpher.pt https://www.dropbox.com/s/8qvo0u5lw7hqvtq/face_morpher.pt?dl=0
# !wget -O data/models/standard_float/two_algo_face_body_rotator.pt https://www.dropbox.com/s/qmq1dnxrmzsxb4h/two_algo_face_body_rotator.pt?dl=0

# !mkdir -p data/models/standard_half
# !wget -O data/models/standard_half/editor.pt https://www.dropbox.com/s/g21ps8gfuvz4kbo/editor.pt?dl=0
# !wget -O data/models/standard_half/eyebrow_decomposer.pt https://www.dropbox.com/s/nwwwevzpmxiilgn/eyebrow_decomposer.pt?dl=0
# !wget -O data/models/standard_half/eyebrow_morphing_combiner.pt https://www.dropbox.com/s/z5v0amgqif7yup1/eyebrow_morphing_combiner.pt?dl=0
# !wget -O data/models/standard_half/face_morpher.pt https://www.dropbox.com/s/g03sfnd5yfs0m65/face_morpher.pt?dl=0
# !wget -O data/models/standard_half/two_algo_face_body_rotator.pt https://www.dropbox.com/s/c5lrn7z34x12317/two_algo_face_body_rotator.pt?dl=0

# !mkdir -p data/models/separable_float 
# !wget -O data/models/separable_float/editor.pt https://www.dropbox.com/s/nwdxhrpa9fy19r4/editor.pt?dl=0
# !wget -O data/models/separable_float/eyebrow_decomposer.pt https://www.dropbox.com/s/hfzjcu9cqr9wm3i/eyebrow_decomposer.pt?dl=0
# !wget -O data/models/separable_float/eyebrow_morphing_combiner.pt https://www.dropbox.com/s/g04dyyyavh5o1e2/eyebrow_morphing_combiner.pt?dl=0
# !wget -O data/models/separable_float/face_morpher.pt https://www.dropbox.com/s/vgi9dsj95y0rrwv/face_morpher.pt?dl=0
# !wget -O data/models/separable_float/two_algo_face_body_rotator.pt https://www.dropbox.com/s/8u0qond8po34l24/two_algo_face_body_rotator.pt?dl=0

# !mkdir -p data/models/separable_half
# !wget -O data/models/separable_half/editor.pt https://www.dropbox.com/s/on8kn6z9fj95j0h/editor.pt?dl=0
# !wget -O data/models/separable_half/eyebrow_decomposer.pt https://www.dropbox.com/s/0hxu8opu1hmghqe/eyebrow_decomposer.pt?dl=0
# !wget -O data/models/separable_half/eyebrow_morphing_combiner.pt https://www.dropbox.com/s/bgz02afp0xojqfs/eyebrow_morphing_combiner.pt?dl=0
# !wget -O data/models/separable_half/face_morpher.pt https://www.dropbox.com/s/bgz02afp0xojqfs/eyebrow_morphing_combiner.pt?dl=0
# !wget -O data/models/separable_half/two_algo_face_body_rotator.pt https://www.dropbox.com/s/vr8h2xxltszhw7w/two_algo_face_body_rotator.pt?dl=0

In [5]:
# Set this constant to specify which system variant to use.
MODEL_NAME = "standard_float" 

# Load the models.
import torch
DEVICE_NAME = 'cuda'
device = torch.device(DEVICE_NAME)

def load_poser(model: str, device: torch.device):
    print("Using the %s model." % model)
    if model == "standard_float":
        from tha3.poser.modes.standard_float import create_poser
        return create_poser(device)
    elif model == "standard_half":
        from tha3.poser.modes.standard_half import create_poser
        return create_poser(device)
    elif model == "separable_float":
        from tha3.poser.modes.separable_float import create_poser
        return create_poser(device)
    elif model == "separable_half":
        from tha3.poser.modes.separable_half import create_poser
        return create_poser(device)
    else:
        raise RuntimeError("Invalid model: '%s'" % model)
        
poser = load_poser(MODEL_NAME, DEVICE_NAME)
poser.get_modules();

Using the standard_float model.
Loading the eyebrow decomposer ... DONE!!!
Loading the eyebrow morphing conbiner ... DONE!!!
Loading the face morpher ... DONE!!!
Loading the face-body rotator ... DONE!!!
Loading the combiner ... DONE!!!


In [6]:
# Create the GUI for manipulating character images.
import PIL.Image
import io
from io import StringIO, BytesIO
import IPython.display
import numpy
import ipywidgets
import time
import threading
import torch
from tha3.util import resize_PIL_image, extract_PIL_image_from_filelike, \
    extract_pytorch_image_from_PIL_image, convert_output_image_from_torch_to_numpy

FRAME_RATE = 30.0

last_torch_input_image = None
torch_input_image = None

def show_pytorch_image(pytorch_image):
    output_image = pytorch_image.detach().cpu()
    numpy_image = numpy.uint8(numpy.rint(convert_output_image_from_torch_to_numpy(output_image) * 255.0))
    pil_image = PIL.Image.fromarray(numpy_image, mode='RGBA')
    IPython.display.display(pil_image)

upload_input_image_button = ipywidgets.FileUpload(
    accept='.png',
    multiple=False,
    layout={
        'width': '512px'
    }
)

output_image_widget = ipywidgets.Output(
    layout={
        'border': '1px solid black',
        'width': '512px',
        'height': '512px'
    }
)

eyebrow_dropdown = ipywidgets.Dropdown(
    options=["troubled", "angry", "lowered", "raised", "happy", "serious"],
    value="troubled",
    description="Eyebrow:",    
)
eyebrow_left_slider = ipywidgets.FloatSlider(
    value=0.0,
    min=0.0,
    max=5.0,
    step=0.01,
    description="Left:",
    readout=True,
    readout_format=".2f"
)
eyebrow_right_slider = ipywidgets.FloatSlider(
    value=0.0,
    min=0.0,
    max=5.0,
    step=0.01,
    description="Right:",
    readout=True,
    readout_format=".2f"
)

eye_dropdown = ipywidgets.Dropdown(
    options=["wink", "happy_wink", "surprised", "relaxed", "unimpressed", "raised_lower_eyelid"],
    value="wink",
    description="Eye:",    
)
eye_left_slider = ipywidgets.FloatSlider(
    value=0.0,
    min=0.0,
    max=5.0,
    step=0.01,
    description="Left:",
    readout=True,
    readout_format=".2f"
)
eye_right_slider = ipywidgets.FloatSlider(
    value=0.0,
    min=0.0,
    max=5.0,
    step=0.01,
    description="Right:",
    readout=True,
    readout_format=".2f"
)

mouth_dropdown = ipywidgets.Dropdown(
    options=["aaa", "iii", "uuu", "eee", "ooo", "delta", "lowered_corner", "raised_corner", "smirk"],
    value="aaa",
    description="Mouth:",    
)
mouth_left_slider = ipywidgets.FloatSlider(
    value=0.0,
    min=0.0,
    max=10.0,
    step=0.01,
    description="Value:",
    readout=True,
    readout_format=".2f"
)
mouth_right_slider = ipywidgets.FloatSlider(
    value=0.0,
    min=0.0,
    max=10.0,
    step=0.01,
    description=" ",
    readout=True,
    readout_format=".2f",
    disabled=True,
)

def update_mouth_sliders(change):
    if mouth_dropdown.value == "lowered_corner" or mouth_dropdown.value == "raised_corner":
        mouth_left_slider.description = "Left:"
        mouth_right_slider.description = "Right:"
        mouth_right_slider.disabled = False
    else:
        mouth_left_slider.description = "Value:"
        mouth_right_slider.description = " "
        mouth_right_slider.disabled = True

mouth_dropdown.observe(update_mouth_sliders, names='value')

iris_small_left_slider = ipywidgets.FloatSlider(
    value=0.0,
    min=0.0,
    max=1.0,
    step=0.01,
    description="Left:",
    readout=True,
    readout_format=".2f"
)
iris_small_right_slider = ipywidgets.FloatSlider(
    value=0.0,
    min=0.0,
    max=1.0,
    step=0.01,
    description="Right:",
    readout=True,
    readout_format=".2f",    
)
iris_rotation_x_slider = ipywidgets.FloatSlider(
    value=0.0,
    min=-1.0,
    max=1.0,
    step=0.01,
    description="X-axis:",
    readout=True,
    readout_format=".2f"
)
iris_rotation_y_slider = ipywidgets.FloatSlider(
    value=0.0,
    min=-1.0,
    max=1.0,
    step=0.01,
    description="Y-axis:",
    readout=True,
    readout_format=".2f",    
)

head_x_slider = ipywidgets.FloatSlider(
    value=0.0,
    min=-1.0,
    max=1.0,
    step=0.01,
    description="X-axis:",
    readout=True,
    readout_format=".2f"
)
head_y_slider = ipywidgets.FloatSlider(
    value=0.0,
    min=-1.0,
    max=1.0,
    step=0.01,
    description="Y-axis:",
    readout=True,
    readout_format=".2f",    
)
neck_z_slider = ipywidgets.FloatSlider(
    value=0.0,
    min=-1.0,
    max=1.0,
    step=0.01,
    description="Z-axis:",
    readout=True,
    readout_format=".2f",    
)
body_y_slider = ipywidgets.FloatSlider(
    value=0.0,
    min=-1.0,
    max=1.0,
    step=0.01,
    description="Y-axis rotation:",
    readout=True,
    readout_format=".2f",    
)
body_z_slider = ipywidgets.FloatSlider(
    value=0.0,
    min=-1.0,
    max=1.0,
    step=0.01,
    description="Z-axis rotation:",
    readout=True,
    readout_format=".2f",    
)
breathing_slider = ipywidgets.FloatSlider(
    value=0.0,
    min=0.0,
    max=5.0,
    step=0.01,
    description="Breathing:",
    readout=True,
    readout_format=".2f",    
)


control_panel = ipywidgets.VBox([
    eyebrow_dropdown,
    eyebrow_left_slider,
    eyebrow_right_slider,
    ipywidgets.HTML(value="<hr>"),
    eye_dropdown,
    eye_left_slider,
    eye_right_slider,
    ipywidgets.HTML(value="<hr>"),
    mouth_dropdown,
    mouth_left_slider,
    mouth_right_slider,
    ipywidgets.HTML(value="<hr>"),
    ipywidgets.HTML(value="<center><b>Iris Shrinkage</b></center>"),
    iris_small_left_slider,
    iris_small_right_slider,
    ipywidgets.HTML(value="<center><b>Iris Rotation</b></center>"),
    iris_rotation_x_slider,
    iris_rotation_y_slider,
    ipywidgets.HTML(value="<hr>"),
    ipywidgets.HTML(value="<center><b>Head Rotation</b></center>"),
    head_x_slider,
    head_y_slider,
    neck_z_slider,
    ipywidgets.HTML(value="<hr>"),
    ipywidgets.HTML(value="<center><b>Body Rotation</b></center>"),
    body_y_slider,
    body_z_slider,
    ipywidgets.HTML(value="<hr>"),
    ipywidgets.HTML(value="<center><b>Breathing</b></center>"),
    breathing_slider,
])

controls = ipywidgets.HBox([
    ipywidgets.VBox([
        output_image_widget, 
        upload_input_image_button
    ]),
    control_panel,
])

from tha3.poser.modes.pose_parameters import get_pose_parameters
pose_parameters = get_pose_parameters()
pose_size = poser.get_num_parameters()
last_pose = torch.zeros(1, pose_size, dtype=poser.get_dtype()).to(device)

iris_small_left_index = pose_parameters.get_parameter_index("iris_small_left")
iris_small_right_index = pose_parameters.get_parameter_index("iris_small_right")
iris_rotation_x_index = pose_parameters.get_parameter_index("iris_rotation_x")
iris_rotation_y_index = pose_parameters.get_parameter_index("iris_rotation_y")
head_x_index = pose_parameters.get_parameter_index("head_x")
head_y_index = pose_parameters.get_parameter_index("head_y")
neck_z_index = pose_parameters.get_parameter_index("neck_z")
body_y_index = pose_parameters.get_parameter_index("body_y")
body_z_index = pose_parameters.get_parameter_index("body_z")
breathing_index = pose_parameters.get_parameter_index("breathing")

def get_pose():
    pose = torch.zeros(1, pose_size, dtype=poser.get_dtype())

    eyebrow_name = f"eyebrow_{eyebrow_dropdown.value}"
    eyebrow_left_index = pose_parameters.get_parameter_index(f"{eyebrow_name}_left")
    eyebrow_right_index = pose_parameters.get_parameter_index(f"{eyebrow_name}_right")
    pose[0, eyebrow_left_index] = eyebrow_left_slider.value
    pose[0, eyebrow_right_index] = eyebrow_right_slider.value

    eye_name = f"eye_{eye_dropdown.value}"
    eye_left_index = pose_parameters.get_parameter_index(f"{eye_name}_left")
    eye_right_index = pose_parameters.get_parameter_index(f"{eye_name}_right")
    pose[0, eye_left_index] = eye_left_slider.value
    pose[0, eye_right_index] = eye_right_slider.value

    mouth_name = f"mouth_{mouth_dropdown.value}"
    if mouth_name == "mouth_lowered_corner" or mouth_name == "mouth_raised_corner":
        mouth_left_index = pose_parameters.get_parameter_index(f"{mouth_name}_left")
        mouth_right_index = pose_parameters.get_parameter_index(f"{mouth_name}_right")
        pose[0, mouth_left_index] = mouth_left_slider.value
        pose[0, mouth_right_index] = mouth_right_slider.value
    else:
        mouth_index = pose_parameters.get_parameter_index(mouth_name)
        pose[0, mouth_index] = mouth_left_slider.value

    pose[0, iris_small_left_index] = iris_small_left_slider.value
    pose[0, iris_small_right_index] = iris_small_right_slider.value
    pose[0, iris_rotation_x_index] = iris_rotation_x_slider.value
    pose[0, iris_rotation_y_index] = iris_rotation_y_slider.value
    pose[0, head_x_index] = head_x_slider.value
    pose[0, head_y_index] = head_y_slider.value
    pose[0, neck_z_index] = neck_z_slider.value
    pose[0, body_y_index] = body_y_slider.value
    pose[0, body_z_index] = body_z_slider.value
    pose[0, breathing_index] = breathing_slider.value

    return pose.to(device)

display(controls)

def update(change):
    global last_pose
    global last_torch_input_image

    if torch_input_image is None:
        return

    needs_update = False
    if last_torch_input_image is None:
        needs_update = True        
    else:
        if (torch_input_image - last_torch_input_image).abs().max().item() > 0:
            needs_update = True         

    pose = get_pose()
    if (pose - last_pose).abs().max().item() > 0:
        needs_update = True

    if not needs_update:
        return

    output_image = poser.pose(torch_input_image, pose)[0]
    with output_image_widget:
        output_image_widget.clear_output(wait=True)
        show_pytorch_image(output_image)  

    last_torch_input_image = torch_input_image
    last_pose = pose

def upload_image(change):
    global torch_input_image
    for name, file_info in upload_input_image_button.value.items():
        content = io.BytesIO(file_info['content'])
    if content is not None:
        pil_image = resize_PIL_image(extract_PIL_image_from_filelike(content), size=(512,512))
        w, h = pil_image.size
        if pil_image.mode != 'RGBA':
            with output_image_widget:
                torch_input_image = None
                output_image_widget.clear_output(wait=True)
                display(ipywidgets.HTML("Image must have an alpha channel!!!"))
        else:
            torch_input_image = extract_pytorch_image_from_PIL_image(pil_image).to(device)
            if poser.get_dtype() == torch.half:
                torch_input_image = torch_input_image.half()
        update(None)

upload_input_image_button.observe(upload_image, names='value')
eyebrow_dropdown.observe(update, 'value')
eyebrow_left_slider.observe(update, 'value')
eyebrow_right_slider.observe(update, 'value')
eye_dropdown.observe(update, 'value')
eye_left_slider.observe(update, 'value')
eye_right_slider.observe(update, 'value')
mouth_dropdown.observe(update, 'value')
mouth_left_slider.observe(update, 'value')
mouth_right_slider.observe(update, 'value')
iris_small_left_slider.observe(update, 'value')
iris_small_right_slider.observe(update, 'value')
iris_rotation_x_slider.observe(update, 'value')

head_x_slider.observe(update, 'value')
head_y_slider.observe(update, 'value')
neck_z_slider.observe(update, 'value')
body_y_slider.observe(update, 'value')
body_z_slider.observe(update, 'value')
breathing_slider.observe(update, 'value')


In [7]:
# Create the GUI for manipulating character images.
import PIL.Image
import io
from io import StringIO, BytesIO
import IPython.display
import numpy
import ipywidgets
import time
import threading
import torch
from tha3.util import resize_PIL_image, extract_PIL_image_from_filelike, \
    extract_pytorch_image_from_PIL_image, convert_output_image_from_torch_to_numpy

FRAME_RATE = 30.0

last_torch_input_image = None
torch_input_image = None

def show_pytorch_image(pytorch_image):
    output_image = pytorch_image.detach().cpu()
    numpy_image = numpy.uint8(numpy.rint(convert_output_image_from_torch_to_numpy(output_image) * 255.0))
    pil_image = PIL.Image.fromarray(numpy_image, mode='RGBA')
    IPython.display.display(pil_image)

upload_input_image_button = ipywidgets.FileUpload(
    accept='.png',
    multiple=False,
    layout={
        'width': '512px'
    }
)

output_image_widget = ipywidgets.Output(
    layout={
        'border': '1px solid black',
        'width': '512px',
        'height': '512px'
    }
)

exp_dropdown = ipywidgets.Dropdown(
    options=["worried", "angry", "angry2", "sad", "happy_wink", "suprised","contempt","happy"],
    value="worried",
    description="Expression:",   
) 
exp_slider = ipywidgets.FloatSlider(
    value=0.0,
    min=0.0,
    max=3.0,
    step=0.01,
    description="Expression:",
    readout=True,
    readout_format=".2f"
)

eyebrow_dropdown = ipywidgets.Dropdown(
    options=["troubled", "angry", "lowered", "raised", "happy", "serious"],
    value="troubled",
    description="Eyebrow:",    
)
eyebrow_left_slider = ipywidgets.FloatSlider(
    value=0.0,
    min=0.0,
    max=5.0,
    step=0.01,
    description="Left:",
    readout=True,
    readout_format=".2f"
)
eyebrow_right_slider = ipywidgets.FloatSlider(
    value=0.0,
    min=0.0,
    max=5.0,
    step=0.01,
    description="Right:",
    readout=True,
    readout_format=".2f"
)

eye_dropdown = ipywidgets.Dropdown(
    options=["wink", "happy_wink", "surprised", "relaxed", "unimpressed", "raised_lower_eyelid"],
    value="wink",
    description="Eye:",    
)
eye_left_slider = ipywidgets.FloatSlider(
    value=0.0,
    min=0.0,
    max=5.0,
    step=0.01,
    description="Left:",
    readout=True,
    readout_format=".2f"
)
eye_right_slider = ipywidgets.FloatSlider(
    value=0.0,
    min=0.0,
    max=5.0,
    step=0.01,
    description="Right:",
    readout=True,
    readout_format=".2f"
)

mouth_dropdown = ipywidgets.Dropdown(
    options=["aaa", "iii", "uuu", "eee", "ooo", "delta", "lowered_corner", "raised_corner", "smirk"],
    value="aaa",
    description="Mouth:",    
)
mouth_left_slider = ipywidgets.FloatSlider(
    value=0.0,
    min=0.0,
    max=10.0,
    step=0.01,
    description="Value:",
    readout=True,
    readout_format=".2f"
)
mouth_right_slider = ipywidgets.FloatSlider(
    value=0.0,
    min=0.0,
    max=10.0,
    step=0.01,
    description=" ",
    readout=True,
    readout_format=".2f",
    disabled=True,
)

def update_mouth_sliders(change):
    if mouth_dropdown.value == "lowered_corner" or mouth_dropdown.value == "raised_corner":
        mouth_left_slider.description = "Left:"
        mouth_right_slider.description = "Right:"
        mouth_right_slider.disabled = False
    else:
        mouth_left_slider.description = "Value:"
        mouth_right_slider.description = " "
        mouth_right_slider.disabled = True

mouth_dropdown.observe(update_mouth_sliders, names='value')

iris_small_left_slider = ipywidgets.FloatSlider(
    value=0.0,
    min=0.0,
    max=1.0,
    step=0.01,
    description="Left:",
    readout=True,
    readout_format=".2f"
)
iris_small_right_slider = ipywidgets.FloatSlider(
    value=0.0,
    min=0.0,
    max=1.0,
    step=0.01,
    description="Right:",
    readout=True,
    readout_format=".2f",    
)
iris_rotation_x_slider = ipywidgets.FloatSlider(
    value=0.0,
    min=-1.0,
    max=1.0,
    step=0.01,
    description="X-axis:",
    readout=True,
    readout_format=".2f"
)
iris_rotation_y_slider = ipywidgets.FloatSlider(
    value=0.0,
    min=-1.0,
    max=1.0,
    step=0.01,
    description="Y-axis:",
    readout=True,
    readout_format=".2f",    
)

head_x_slider = ipywidgets.FloatSlider(
    value=0.0,
    min=-1.0,
    max=1.0,
    step=0.01,
    description="X-axis:",
    readout=True,
    readout_format=".2f"
)
head_y_slider = ipywidgets.FloatSlider(
    value=0.0,
    min=-1.0,
    max=1.0,
    step=0.01,
    description="Y-axis:",
    readout=True,
    readout_format=".2f",    
)
neck_z_slider = ipywidgets.FloatSlider(
    value=0.0,
    min=-1.0,
    max=1.0,
    step=0.01,
    description="Z-axis:",
    readout=True,
    readout_format=".2f",    
)
body_y_slider = ipywidgets.FloatSlider(
    value=0.0,
    min=-1.0,
    max=1.0,
    step=0.01,
    description="Y-axis rotation:",
    readout=True,
    readout_format=".2f",    
)
body_z_slider = ipywidgets.FloatSlider(
    value=0.0,
    min=-1.0,
    max=1.0,
    step=0.01,
    description="Z-axis rotation:",
    readout=True,
    readout_format=".2f",    
)
breathing_slider = ipywidgets.FloatSlider(
    value=0.0,
    min=0.0,
    max=1.0,
    step=0.01,
    description="Breathing:",
    readout=True,
    readout_format=".2f",    
)


control_panel = ipywidgets.VBox([
    exp_dropdown,
    exp_slider,
    ipywidgets.HTML(value="<hr>"),
    eyebrow_dropdown,
    eyebrow_left_slider,
    eyebrow_right_slider,
    ipywidgets.HTML(value="<hr>"),
    eye_dropdown,
    eye_left_slider,
    eye_right_slider,
    ipywidgets.HTML(value="<hr>"),
    mouth_dropdown,
    mouth_left_slider,
    mouth_right_slider,
    ipywidgets.HTML(value="<hr>"),
    ipywidgets.HTML(value="<center><b>Iris Shrinkage</b></center>"),
    iris_small_left_slider,
    iris_small_right_slider,
    ipywidgets.HTML(value="<center><b>Iris Rotation</b></center>"),
    iris_rotation_x_slider,
    iris_rotation_y_slider,
    ipywidgets.HTML(value="<hr>"),
    ipywidgets.HTML(value="<center><b>Head Rotation</b></center>"),
    head_x_slider,
    head_y_slider,
    neck_z_slider,
    ipywidgets.HTML(value="<hr>"),
    ipywidgets.HTML(value="<center><b>Body Rotation</b></center>"),
    body_y_slider,
    body_z_slider,
    ipywidgets.HTML(value="<hr>"),
    ipywidgets.HTML(value="<center><b>Breathing</b></center>"),
    breathing_slider,
])

controls = ipywidgets.HBox([
    ipywidgets.VBox([
        output_image_widget, 
        upload_input_image_button
    ]),
    control_panel,
])

from tha3.poser.modes.pose_parameters import get_pose_parameters
pose_parameters = get_pose_parameters()
pose_size = poser.get_num_parameters()
last_pose = torch.zeros(1, pose_size, dtype=poser.get_dtype()).to(device)

iris_small_left_index = pose_parameters.get_parameter_index("iris_small_left")
iris_small_right_index = pose_parameters.get_parameter_index("iris_small_right")
iris_rotation_x_index = pose_parameters.get_parameter_index("iris_rotation_x")
iris_rotation_y_index = pose_parameters.get_parameter_index("iris_rotation_y")
head_x_index = pose_parameters.get_parameter_index("head_x")
head_y_index = pose_parameters.get_parameter_index("head_y")
neck_z_index = pose_parameters.get_parameter_index("neck_z")
body_y_index = pose_parameters.get_parameter_index("body_y")
body_z_index = pose_parameters.get_parameter_index("body_z")
breathing_index = pose_parameters.get_parameter_index("breathing")

# def update_exp(control):
#     global last_pose

#     if torch_input_image is None:
#         return
#     print(control)
#     if control['name'] == 'type':
#       pose = get_pose_exp(control['new'], 1.)
#     else:
#       pose = get_pose_exp(None, control['new'])


#     output_image = poser.pose(torch_input_image, pose)[0]
#     with output_image_widget:
#         output_image_widget.clear_output(wait=True)
#         show_pytorch_image(output_image)  

#     last_pose = pose
#     return

def update_exp(control):
    global last_pose

    if torch_input_image is None:
        return
    print(control)
    if control['type'] == 'change':
      exp_type = exp_dropdown.value
      pose = get_pose_exp(exp_type, control['new'])
    else:
      pose = get_pose_exp(None, control['new'])


    output_image = poser.pose(torch_input_image, pose)[0]
    with output_image_widget:
        output_image_widget.clear_output(wait=True)
        show_pytorch_image(output_image)  

    last_pose = pose
    return
# def get_pose_exp(exp_type, value):
#     pose = torch.ones(1, pose_size, dtype=poser.get_dtype()) * value
#     return pose.to(device)

def get_pose_exp(exp_type, value):
    pose = torch.zeros(1, pose_size, dtype=poser.get_dtype())
    print(exp_type)
    if exp_type == "worried":
      eyebrow_dropdown.value = "troubled"
      eye_dropdown.value = "raised_lower_eyelid"
      mouth_dropdown.value = "ooo"
      eyebrow_left_slider.value = 1.00 * value
      eyebrow_right_slider.value = 1.00 * value
      eye_left_slider.value = 0.30 * value
      eye_right_slider.value = 0.30 * value
      mouth_left_slider.value = 3.00 * value

    elif exp_type == "angry":
      eyebrow_dropdown.value = "angry"
      eye_dropdown.value = "surprised"
      mouth_dropdown.value = "delta"
      eyebrow_left_slider.value = 1.90 * value
      eyebrow_right_slider.value = 1.80 * value
      eye_left_slider.value = 3.55 * value
      eye_right_slider.value = 3.55 * value
      mouth_left_slider.value = 8.00 * value

    elif exp_type == "angry2":
      eyebrow_dropdown.value = "angry"
      eye_dropdown.value = "surprised"
      mouth_dropdown.value = "lowered_corner"
      eyebrow_left_slider.value = 1.90 * value
      eyebrow_right_slider.value = 1.80 * value
      eye_left_slider.value = 3.55 * value
      eye_right_slider.value = 3.55 * value
      mouth_left_slider.value = 1.50 * value
      mouth_right_slider.value = 0.0 * value
    
    elif exp_type == "suprised":
      eyebrow_dropdown.value = "raised"
      eye_dropdown.value = "surprised"
      mouth_dropdown.value = "ooo"
      eyebrow_left_slider.value = 1.50 * value
      eyebrow_right_slider.value = 1.50 * value
      eye_left_slider.value = 1.23 * value
      eye_right_slider.value = 1.23 * value
      mouth_left_slider.value = 1.45 * value
    
    elif exp_type == "contempt":
      eyebrow_dropdown.value = "angry"
      eye_dropdown.value = "unimpressed"
      mouth_dropdown.value = "delta"
      eyebrow_left_slider.value = 1.50 * value
      eyebrow_right_slider.value = 1.40 * value
      eye_left_slider.value = 0.80 * value
      eye_right_slider.value = 0.80 * value
      mouth_left_slider.value = 1.05 * value
    
    elif exp_type == "happy_wink":
      eyebrow_dropdown.value = "happy"
      eye_dropdown.value = "happy_wink"
      mouth_dropdown.value = "ooo"
      eyebrow_left_slider.value = 0.80 * value
      eyebrow_right_slider.value = 0.80 * value
      eye_left_slider.value = 1.00 * value
      eye_right_slider.value = 1.00 * value
      mouth_left_slider.value = 0.48 * value
    
    elif exp_type == "sad":
      eyebrow_dropdown.value = "troubled"
      eye_dropdown.value = "raised_lower_eyelid"
      mouth_dropdown.value = "aaa"
      eyebrow_left_slider.value = 0.38 * value
      eyebrow_right_slider.value = 0.41 * value
      eye_left_slider.value = 0.42 * value
      eye_right_slider.value = 0.45 * value
      mouth_left_slider.value = 0.50 * value

    elif exp_type == "happy":
      eyebrow_dropdown.value = "happy"
      eye_dropdown.value = "surprised"
      mouth_dropdown.value = "raised_corner"
      eyebrow_left_slider.value = 0.80 * value
      eyebrow_right_slider.value = 0.80 * value
      eye_left_slider.value = 0.10 * value
      eye_right_slider.value = 0.10 * value
      mouth_left_slider.value = 1.50 * value
      mouth_right_slider.value = 1.50 * value


    
    eyebrow_name = f"eyebrow_{eyebrow_dropdown.value}"
    eyebrow_left_index = pose_parameters.get_parameter_index(f"{eyebrow_name}_left")
    eyebrow_right_index = pose_parameters.get_parameter_index(f"{eyebrow_name}_right")
    pose[0, eyebrow_left_index] = eyebrow_left_slider.value
    pose[0, eyebrow_right_index] = eyebrow_right_slider.value

    eye_name = f"eye_{eye_dropdown.value}"
    eye_left_index = pose_parameters.get_parameter_index(f"{eye_name}_left")
    eye_right_index = pose_parameters.get_parameter_index(f"{eye_name}_right")
    pose[0, eye_left_index] = eye_left_slider.value
    pose[0, eye_right_index] = eye_right_slider.value

    mouth_name = f"mouth_{mouth_dropdown.value}"
    if mouth_name == "mouth_lowered_corner" or mouth_name == "mouth_raised_corner":
        mouth_left_index = pose_parameters.get_parameter_index(f"{mouth_name}_left")
        mouth_right_index = pose_parameters.get_parameter_index(f"{mouth_name}_right")
        pose[0, mouth_left_index] = mouth_left_slider.value
        pose[0, mouth_right_index] = mouth_right_slider.value
    else:
        mouth_index = pose_parameters.get_parameter_index(mouth_name)
        pose[0, mouth_index] = mouth_left_slider.value

    return pose.to(device)

def get_pose():
    pose = torch.zeros(1, pose_size, dtype=poser.get_dtype())

    eyebrow_name = f"eyebrow_{eyebrow_dropdown.value}"
    eyebrow_left_index = pose_parameters.get_parameter_index(f"{eyebrow_name}_left")
    eyebrow_right_index = pose_parameters.get_parameter_index(f"{eyebrow_name}_right")
    pose[0, eyebrow_left_index] = eyebrow_left_slider.value
    pose[0, eyebrow_right_index] = eyebrow_right_slider.value

    eye_name = f"eye_{eye_dropdown.value}"
    eye_left_index = pose_parameters.get_parameter_index(f"{eye_name}_left")
    eye_right_index = pose_parameters.get_parameter_index(f"{eye_name}_right")
    pose[0, eye_left_index] = eye_left_slider.value
    pose[0, eye_right_index] = eye_right_slider.value

    mouth_name = f"mouth_{mouth_dropdown.value}"
    if mouth_name == "mouth_lowered_corner" or mouth_name == "mouth_raised_corner":
        mouth_left_index = pose_parameters.get_parameter_index(f"{mouth_name}_left")
        mouth_right_index = pose_parameters.get_parameter_index(f"{mouth_name}_right")
        pose[0, mouth_left_index] = mouth_left_slider.value
        pose[0, mouth_right_index] = mouth_right_slider.value
    else:
        mouth_index = pose_parameters.get_parameter_index(mouth_name)
        pose[0, mouth_index] = mouth_left_slider.value

    pose[0, iris_small_left_index] = iris_small_left_slider.value
    pose[0, iris_small_right_index] = iris_small_right_slider.value
    pose[0, iris_rotation_x_index] = iris_rotation_x_slider.value
    pose[0, iris_rotation_y_index] = iris_rotation_y_slider.value
    pose[0, head_x_index] = head_x_slider.value
    pose[0, head_y_index] = head_y_slider.value
    pose[0, neck_z_index] = neck_z_slider.value
    pose[0, body_y_index] = body_y_slider.value
    pose[0, body_z_index] = body_z_slider.value
    pose[0, breathing_index] = breathing_slider.value

    return pose.to(device)

display(controls)

def update(change):
    global last_pose
    global last_torch_input_image

    if torch_input_image is None:
        return

    needs_update = False
    if last_torch_input_image is None:
        needs_update = True        
    else:
        if (torch_input_image - last_torch_input_image).abs().max().item() > 0:
            needs_update = True         

    pose = get_pose()
    if (pose - last_pose).abs().max().item() > 0:
        needs_update = True

    if not needs_update:
        return

    output_image = poser.pose(torch_input_image, pose)[0]
    with output_image_widget:
        output_image_widget.clear_output(wait=True)
        show_pytorch_image(output_image)  

    last_torch_input_image = torch_input_image
    last_pose = pose

def upload_image(change):
    global torch_input_image
    for name, file_info in upload_input_image_button.value.items():
        content = io.BytesIO(file_info['content'])
    if content is not None:
        pil_image = resize_PIL_image(extract_PIL_image_from_filelike(content), size=(512,512))
        w, h = pil_image.size
        if pil_image.mode != 'RGBA':
            with output_image_widget:
                torch_input_image = None
                output_image_widget.clear_output(wait=True)
                display(ipywidgets.HTML("Image must have an alpha channel!!!"))
        else:
            torch_input_image = extract_pytorch_image_from_PIL_image(pil_image).to(device)
            if poser.get_dtype() == torch.half:
                torch_input_image = torch_input_image.half()
        update(None)

upload_input_image_button.observe(upload_image, names='value')
exp_dropdown.observe(update_exp, 'type')
exp_slider.observe(update_exp, 'value')
eyebrow_dropdown.observe(update, 'value')
eyebrow_left_slider.observe(update, 'value')
eyebrow_right_slider.observe(update, 'value')
eye_dropdown.observe(update, 'value')
eye_left_slider.observe(update, 'value')
eye_right_slider.observe(update, 'value')
mouth_dropdown.observe(update, 'value')
mouth_left_slider.observe(update, 'value')
mouth_right_slider.observe(update, 'value')
iris_small_left_slider.observe(update, 'value')
iris_small_right_slider.observe(update, 'value')
iris_rotation_x_slider.observe(update, 'value')
iris_rotation_y_slider.observe(update, 'value')
head_x_slider.observe(update, 'value')
head_y_slider.observe(update, 'value')
neck_z_slider.observe(update, 'value')
body_y_slider.observe(update, 'value')
body_z_slider.observe(update, 'value')
breathing_slider.observe(update, 'value')

{'name': 'value', 'old': 0.0, 'new': 0.68, 'owner': FloatSlider(value=0.68, description='Expression:', max=3.0, step=0.01), 'type': 'change'}
worried
{'name': 'value', 'old': 0.68, 'new': 1.36, 'owner': FloatSlider(value=1.36, description='Expression:', max=3.0, step=0.01), 'type': 'change'}
worried
{'name': 'value', 'old': 1.36, 'new': 1.0, 'owner': FloatSlider(value=1.0, description='Expression:', max=3.0, step=0.01), 'type': 'change'}
sad
{'name': 'value', 'old': 1.0, 'new': 0.97, 'owner': FloatSlider(value=0.97, description='Expression:', max=3.0, step=0.01), 'type': 'change'}
sad
{'name': 'value', 'old': 0.97, 'new': 3.0, 'owner': FloatSlider(value=3.0, description='Expression:', max=3.0, step=0.01), 'type': 'change'}
sad
{'name': 'value', 'old': 3.0, 'new': 2.5, 'owner': FloatSlider(value=2.5, description='Expression:', max=3.0, step=0.01), 'type': 'change'}
suprised
{'name': 'value', 'old': 2.5, 'new': 2.69, 'owner': FloatSlider(value=2.69, description='Expression:', max=3.0, st